# Locally simultaneous inference for the LASSO

We study the problem of constructing confidence intervals for the OLS regression solution after model selection via the LASSO. We compare conditional inference due to Lee et al. [1], hybrid inference due to McCloskey [2], the PoSI method for simultaneous inference due to Berk et al. [3], and locally simultaneous inference.

[1] Lee, J. D., Sun, D. L., Sun, Y., & Taylor, J. E. (2016). Exact post-selection inference, with application to the lasso. Annals of Statistics 44(3), 907-927.

[2] Andrews, I., Kitagawa, T., & McCloskey, A. (2020). Hybrid confidence intervals for informative uniform asymptotic inference after model selection. Forthcoming at Biometrika.

[3] Berk, R., Brown, L., Buja, A., Zhang, K., & Zhao, L. (2013). Valid post-selection inference. Annals of Statistics, 802-837.

In [ ]:
%load_ext autoreload
%autoreload 2
from scipy.optimize import linprog
import numpy as np
from numpy.linalg import inv
from sklearn import linear_model
import scipy
import math
import matplotlib.pyplot as plt

from methods import *
from utils import *
from lasso_utils import *

## Varying sparsity in high dimensions

We vary the dimension $d$ between 40 and 100. We illustrate how, even though the PoSI method is not applicable for such large $d$, locally simultaneous inference is still applicable if there are not too many plausible models. We compare the intervals to conditional intervals of Lee et al. We do not compute the hybrid intervals as that would require computing simultaneous intervals as an intermediate step, which, again, is infeasible for large $d$.

In [ ]:
# simulation params
trials = 100
alpha = 0.1
nu = 0.01
sparsity = [0.05, 0.1]
frac_weak_entries = 0.5

dims = np.linspace(40,80,3)
n = 1000

for s in sparsity:
    
    cond_widths = np.zeros((len(dims), trials))
    hybrid_widths = np.zeros((len(dims), trials))
    local_widths = np.zeros((len(dims), trials))
    simultaneous_widths = np.zeros((len(dims), trials))
    
    for j in range(len(dims)):
        d = round(dims[j])
        lam = 6*np.sqrt(2*np.log(np.e*d))
        sig_weak = lam
        sig_strong = 2*lam
        num_strong_entries = math.ceil(d*s*(1-frac_weak_entries))
        num_weak_entries = math.ceil(d*s*frac_weak_entries)
        beta_strong = np.ones(num_strong_entries)*sig_strong
        beta_weak = np.ones(num_weak_entries)*sig_weak
        beta_null = np.zeros(d - num_strong_entries - num_weak_entries)
        beta = np.concatenate((beta_strong,beta_weak,beta_null), axis=0)

        for i in range(trials):

            print("Trial {}, dimension {}, sparsity {}".format(i,d,s))

            X, y = generate_data(n, d, beta)

            # fit LASSO on realized data
            lasso_model = linear_model.Lasso(alpha=lam/n, fit_intercept=False)
            beta_y = lasso_model.fit(X,y).coef_
            M_y = [i for i in range(d) if beta_y[i]]
            print(M_y)
            s_y = np.sign(beta_y[M_y])

            # locally simultaneous inference
            Ms_pairs = plausible_LASSO_models_and_signs(n, d, lam, X, y, nu, M_y, s_y)
            local_ints = locally_simultaneous_LASSO(X, y, Ms_pairs, M_y, alpha=alpha, nu=nu)
            local_widths[j,i] = np.mean(local_ints[1] - local_ints[0])
            
            # conditional inference
            X_M, X_Mc, E_M, E_Mc = X_in_selected_model(X, M_y)

            # LASSO constraints
            A0_plus, A0_minus, A1, b0_plus, b0_minus, b1 = lasso_constraints_Xy_space(lam, X_Mc, X_M, E_M, E_Mc, s_y)
            A = np.concatenate((A0_plus, A0_minus, A1))
            b = np.concatenate((b0_plus, b0_minus, b1))
            
            X_M_pinv = np.linalg.pinv(X_M)
            cond_ints = [np.zeros(len(M_y)), np.zeros(len(M_y))]

            for k in range(len(M_y)):
                eta = X_M_pinv[k,:]
                [cond_ints[0][k], cond_ints[1][k]] = conditional_inference(y, np.eye(n), A, b, eta, alpha/len(M_y))
            cond_widths[j,i] = np.mean(cond_ints[1] - cond_ints[0])
            
    plot_title = 's = ' + str(s)
    ylabel = 'interval width'
    xlabel = 'd'
    filename = 'LASSO_inf_sparsity' + str(s) + '.pdf'
    plot_and_save_results(dims, simultaneous_widths, local_widths, cond_widths, hybrid_widths, plot_title, ylabel, xlabel, filename, plot_hybrid=False, plot_simultaneous=False)

## Varying signal strength

In [ ]:
# simulation params
trials = 100
alpha = 0.1
nu = 0.01
sig_strength = np.linspace(1,6,6)
ratio_eps_to_lam = [1, 1.5, 2]
d = 10
n = 1000


for r in ratio_eps_to_lam:
    
    cond_widths = np.zeros((len(sig_strength), trials))
    hybrid_widths = np.zeros((len(sig_strength), trials))
    local_widths = np.zeros((len(sig_strength), trials))
    simultaneous_widths = np.zeros((len(sig_strength), trials))
    
    for j in range(len(sig_strength)):
        lam = sig_strength[j]*np.sqrt(2*np.log(np.e*d))
        sig = r*lam
        
        beta = np.zeros(d)
        beta[:5] = sig

        for i in range(trials):
            print("Trial {}, signal {}".format(i,sig_strength[j]))

            X, y = generate_data(n, d, beta)

            # fit LASSO on realized data
            lasso_model = linear_model.Lasso(alpha=lam/n, fit_intercept=False)
            beta_y = lasso_model.fit(X,y).coef_
            M_y = [i for i in range(d) if beta_y[i]]
            s_y = np.sign(beta_y[M_y])
#             print(M_y)

                
            # locally simultaneous inference
            Ms_pairs = plausible_LASSO_models_and_signs(n, d, lam, X, y, nu, M_y, s_y)
            local_ints = locally_simultaneous_LASSO(X, y, Ms_pairs, M_y, alpha=alpha, nu=nu)
            local_widths[j,i] = np.mean(local_ints[1] - local_ints[0])
            
            # fully simultaneous inference
            simultaneous_ints = simultaneous_PoSI(X, y, list(powerset(range(d)))[1:], M_y, alpha=alpha)
            simultaneous_widths[j,i] = np.mean(simultaneous_ints[1] - simultaneous_ints[0])

            if len(M_y) == 0:
                cond_widths[j,i] = 0
                hybrid_widths[j,i] = 0
                continue

            # conditional and hybrid inference
            X_M, X_Mc, E_M, E_Mc = X_in_selected_model(X, M_y)

            # LASSO constraints
            A0_plus, A0_minus, A1, b0_plus, b0_minus, b1 = lasso_constraints_Xy_space(lam, X_Mc, X_M, E_M, E_Mc, s_y)
            A = np.concatenate((A0_plus, A0_minus, A1))
            b = np.concatenate((b0_plus, b0_minus, b1))
            
            # simultaneous projection for hybrid
            simultaneous_proj = simultaneous_PoSI(X, y, list(powerset(range(d)))[1:], M_y, alpha=nu)
            
            X_M_pinv = np.linalg.pinv(X_M)
            cond_ints = [np.zeros(len(M_y)), np.zeros(len(M_y))]
            hybrid_ints = [np.zeros(len(M_y)), np.zeros(len(M_y))]

            for k in range(len(M_y)):
                eta = X_M_pinv[k,:]
                SI_truncation = (simultaneous_proj[1][k] - simultaneous_proj[0][k])/2
                [cond_ints[0][k], cond_ints[1][k]] = conditional_inference(y, np.eye(n), A, b, eta, alpha=alpha/len(M_y))
                [hybrid_ints[0][k], hybrid_ints[1][k]] = hybrid_inference(y, np.eye(n), A, b, eta, alpha=alpha/len(M_y), beta=nu/len(M_y), SI_halfwidth=SI_truncation)
            cond_widths[j,i] = np.mean(cond_ints[1] - cond_ints[0])
            hybrid_widths[j,i] = np.mean(hybrid_ints[1] - hybrid_ints[0])
            
            
    plot_title = 'ε/λ = ' + str(r)
    ylabel = 'interval width'
    xlabel = r'λ₀'
    filename = 'LASSO_inf_ratio' + str(r) + '.pdf'
    plot_and_save_results(sig_strength, simultaneous_widths, local_widths, cond_widths, hybrid_widths, plot_title, ylabel, xlabel, filename)